# Modellierung: Regressions- und Klassifikationsmodelle auf Finanzdaten

In diesem Notebook werden verschiedene Modelle auf einem Finanzdatensatz angewendet. Ziel ist es, sowohl Regressions- als auch Klassifikationsmodelle zu erstellen und zu evaluieren. Folgende Modelle werden betrachtet:

- **Multiple Lineare Regression (MLR):** Vorhersage der absoluten Rendite (|Return|)

- **Naive Bayes:** Klassifikation der Kursrichtung (steigend/fallend)

- **Lineare Diskriminanzanalyse (LDA):** Klassifikation der Kursrichtung



Das Notebook ist in mehrere Abschnitte gegliedert, die jeweils durch Markdown-Zellen erläutert werden.

## **1. Abhängikeiten laden & Daten einlesen**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from utils.modeling.models import mlr, naive_bayes, lda, logistic_regression_classifier
from utils.modeling.evaluation import evaluate_regression, evaluate_classification

df = pd.read_csv('data/merged/merged_by_timestamp.csv')
display(df.head())
display(df.tail())


Wichtige Uhrzeiten übernehmen

## **2. Datenfilterung**

Hier wird eine Maske erstellt welche die ```top_times```, die Top Uhrzeiten geflitert im [02_eda.ipynb](02_eda.ipynb), erschließt.

Die Maske besteht aus folgendn Uhrzeiten:
- 13:30:00
- 15:00:00
- 19:00:00

Dadurch konzentrieren wir uns auf Zeitpunkte, an denen relevante Ereignisse im Datensatz vorliegen.

In [ ]:
top_times = ["13:30:00", "15:00:00", "19:00:00"]

mask_times   = df["Timestamp"].str.endswith(tuple(top_times))
mask_events  = df["event_count"] > 0
mask         = mask_times & mask_events

display(df.loc[mask].head())
display(df.loc[mask].tail())

## **3. Feature-Engineering**
Erstellt verschobene Features und berechnet prozentuale Preisänderungen.

In [ ]:
df["Volume_shifted"]         = df["Volume"].shift(1)
df["|Return|_shifted1"]      = df["|Return|"].shift(1)
df["|Close_pct_change_2h|"]  = ((df["Close"].shift(1) - df["Close"].shift(5)) / df["Close"].shift(5) * 100).abs()
df["|Close_pct_change_4h|"]  = ((df["Close"].shift(1) - df["Close"].shift(9)) / df["Close"].shift(5) * 100).abs()


mask &= df["Volume_shifted"].notna()

print("shape nach Maskierung:", df.loc[mask].shape)

## **4. Zielvariablen und Features**
Definiert die Targets und entfernt nicht benötigte Spalten.


In [ ]:
df["Label"] = (df["Return"] >= 0).astype(int)

drop_columns = [
    "Open", "High", "Low", "Close",
    "Return", "|Return|", "Volume",
    "Timestamp", "Label"
]

X = df.loc[mask].drop(columns=drop_columns, errors="ignore")
print("Anzahl der Datenpunkte in X:", len(X))

y_regression     = df.loc[mask, "|Return|"].values
y_classification = df.loc[mask, "Label"].values


## **5. Regression mit Bootstrap-Intervallen**

In [ ]:
model1, X_tr, X_te, y_tr, y_te = mlr(
    X, y_regression,
    train_ratio=0.5,
    random_state=42,
    n_bootstrap=10_000,
    ci=0.95
)

evaluate_regression(model1, X_tr, y_tr, X_te, y_te, n_last=50)


### Interpretation
*Prognosemodell: absoluter, prozentualer 30-Min-Return nach News-Veröffentlichung*

##### 1 ) Güte der Gesamt­gleichung  

| Kennzahl | Train | Test | Interpretation |
|----------|------:|-----:|----------------|
| **R²**   | 0.269 | 0.258 | ≈ 26 % der Varianz wird erklärt – bei hoch­frequenten Finanz­daten beachtlich, aber das meiste bleibt Rauschen. |
| **RMSE** | 0.208 | 0.206 | Fehler liegt in ähnlicher Größen­ordnung wie der mittlere Target-Wert. |
| **MAE**  | 0.145 | 0.140 | Robustes Fehler­maß bestätigt RMSE-Befund. |
| **Train vs. Test** | – | – | Fast identische Ergebnisse ⇒ kaum Overfitting; lineares Modell stößt schlicht an Grenzen. |


##### 2 ) Bedeutung der Koeffizienten (Bootstrap-CIs)  

| Feature (Auswahl) | β | 95 %-CI | Deutung |
|-------------------|---:|:-------:|---------|
| `\|Return\|_shifted1` | **0.25** | [+0.17; +0.35] | Volatility-Clustering: vergangene Schwankung setzt sich fort. |
| `cat_interest_rate_impact_max` | **0.03** | [+0.01; +0.05] | Zins-Schlagzeilen erhöhen Volatilität signifikant. |
| `cat_labor_market_impact_max` | **-0.03** | [-0.04; -0.01] | Arbeitsmarkt-News senken Volatilität – Reaktion oft vorab eingepreist. |
| `impact_max` | 0.15 | [-0.01; +0.32] | Grenzwertig positiv: einzelne „laute“ Meldungen wirken wohl. |
| Weitere Aggregats- & Preis­features | – | CI überlappt 0 | Kein robuster Einfluss → möglicher Feature-Overload. |



##### 3 ) Prognose­intervalle & Fehlerbild  

*Plot-Beobachtungen*  
- Blaue Fläche deckt die meisten Punkte → PIs sind brauchbar.  
- Extreme Returns (≈ 1.5 – 1.8 %) liegen komplett außerhalb ⇒ Varianz nicht konstant (Heavy Tails).  
- Vorhersage­linie glättet Peaks – typisch MSE-optimiertes lineares Modell.


##### 4 ) TL;DR  

- Modell erklärt **≈ 26 %** der 30-Min-Volatilität – solide für einen linearen Ansatz.  
- Signifikanter Nutzen: **Vorperioden-Volatilität** & starke **Zins/Arbeitsmarkt-News**.  
- **Generalisiert** ordentlich, überfit­tet nicht, wird aber durch Rauschen & fette Schwänze begrenzt.  
- Mehr Power erfordert **nicht-lineare Modelle** und **bessere Fehler­modellierung**.

---


## **6. Gewichtskonstruktion**

Die Gewichte basieren auf dem einfachen Verhältnis

**ρ = Mittelwert mit News / Mittelwert Baseline**

für jede News-Kategorie.  
Alle ρ-Werte sind positiv und können daher direkt als Gewichte in einer
gewichteten linearen Regression verwendet werden.

### Input-Daten und abgeleitete Gewichte

| Kategorie           | Mittelwert mit News | Mittelwert Baseline | Gewicht ρ (News / Baseline) |
|---------------------|--------------------:|--------------------:|----------------------------:|
| politics            | 0.1612 | 0.1734 | 0.9296 |
| interest_rate       | 0.6288 | 0.1734 | 3.6262 |
| labor_market        | 0.3527 | 0.1734 | 2.0340 |
| inflation           | 0.3594 | 0.1734 | 2.0726 |
| central_banks       | 0.2886 | 0.1734 | 1.6643 |
| economic_activity   | 0.2885 | 0.1734 | 1.6637 |


In [ ]:
weight_map = {
    "politics":          0.9296,
    "interest_rate":     3.6262,
    "labor_market":      2.0340,
    "inflation":         2.0726,
    "central_banks":     1.6643,
    "economic_activity": 1.6637,
}

X["weighted_col"] = X.apply(
    lambda row: max([weight_map[c] for c in weight_map if row[f"cat_{c}_event_count"] > 0], default=0),
    axis=1
)

## **7. Gewichtete Multiple Lineare Regression (MLR)**

Es wird eine multiple lineare Regression auf die absolute Rendite (|Return|) durchgeführt, um Zusammenhänge zwischen den Features und der Zielgröße zu untersuchen.

In [ ]:
model2, X_train2, X_test2, y_train2, y_test2 = mlr(
    X, y_regression,
    train_ratio=0.5,
    random_state=42,
    n_bootstrap=10_000,
    ci=0.95,
    weight="weighted_col"
)
evaluate_regression(model2, X_train2, y_train2, X_test2, y_test2)


### Interpretation 

##### 0 ) Kurzüberblick  

| Modell | Gewichtung | R² (Test) | RMSE (Test) | MAE (Test) |
|--------|-----------|-----------|-------------|------------|
| **Ungewichtet** | – | **0.258** | 0.206 | 0.140 |
| **Gewichtet**   | ✓ | **0.255** | 0.206 | 0.141 |

> **Take-away:** Die Gewichtung verändert die Gesamtgüte **kaum** – alle Fehler­metriken bleiben praktisch identisch.  

##### 1 ) Güte der gewichteten Regression  

| Kennzahl | Train | Test | Interpretation |
|----------|------:|-----:|----------------|
| **R²**   | 0.268 | 0.255 | ≈ 25 % erklärte Varianz – identisch zum ungewichteten Modell. |
| **RMSE** | 0.208 | 0.206 | Fehlergröße weiterhin in derselben Größenordnung wie der Zielwert selbst. |
| **MAE**  | 0.146 | 0.141 | Spiegelt RMSE-Befund; kein Overfitting sichtbar (Train≈Test). |

##### 2 ) Feature-Effekte (Bootstrap-CIs, gewichtetes Modell)  

| Feature (Auswahl) | β | 95 %-CI | Deutung |
|-------------------|---:|:-------:|---------|
| `\|Return\|_shifted1` | **0.27** | [+0.18; +0.37] | Volatility-Clustering; Effekt **größer** als im Basis-Modell (+0.25). |
| `cat_interest_rate_event_count` | **0.011** | [+0.004; +0.018] | *Neu signifikant*: Schon die **Anzahl** von Zins-Meldungen treibt Volatilität. |
| `cat_interest_rate_impact_sum` | **0.032** | [+0.011; +0.055] | Bleibt klar positiv; Gewichtung verstärkt den Effekt leicht. |
| `impact_max` | 0.15 | [−0.02; +0.33] | Grenzwertig, ähnlich wie Basis-Modell. |
| `cat_labor_market_impact_sum` | 0.018 | [−0.006; +0.045] | Nicht mehr signifikant (überlappt 0) – Gewichtung **schwächt** diesen Einfluss. |
| `Volume_shifted` | ~0 | CI enthält 0 | Weiterhin vernachlässigbar. |
| *Restliche Aggregats-/Preis-Features* | – | CI überlappt 0 | Kein robuster Einfluss – unverändert. |

**Was hat sich geändert?**  
- **Zinsrelevante Variablen** gewinnen an Bedeutung (Event-Count & Impact-Sum beide signifikant).  
- **Arbeitsmarkt-Signale** verlieren Signifikanz.  
- Gesamte Varianzaufklärung bleibt jedoch gleich – die Gewichtung verschiebt **relativ** die Wichtigkeit einzelner Prädiktoren, nicht aber die Gesamtleistung.

##### 3 ) Prognoseintervalle & Fehlerbild  

*Visualeinschätzung (identisch zu Basis-Plot)*  
- 95 %-Intervalle decken die meisten tatsächlichen Werte ab.  
- Die extremsten Returns (> 1.5 %) liegen weiterhin außerhalb ⇒ Heavy-Tail-Problem bleibt ungelöst.  
- Keine sichtbare systematische Verzerrung durch die Gewichtung.

##### 4 ) Interpretation des Gewichtungs­effekts  

| Beobachtung | Mögliche Ursache |
|-------------|------------------|
| **Stabile Gesamtmetriken** | Gewicht scheinen v. a. Beobachtungen zu betreffen, die bereits im linearen Rahmen gut erklärbar sind; sie ändern nicht das inhärente Rausch-/Signal-Verhältnis. |
| **Stärkere Zinssensitivität** | Gewichtung könnte Perioden mit makro-relevanten Bewegungen (z. B. FOMC-Tage) höher bewertet haben. |
| **Abnehmende Arbeitsmarkt-Relevanz** | Entweder geringeres Gewicht auf NFP-Tage oder Überbetonung anderer Event-Typen. |

##### 5 ) TL;DR  

- **Leistungs­gleichstand**: Gewichtung ändert die Modellgüte **nicht** spürbar.  
- **Signalverschiebung**: Zinsspezifische Features werden wichtiger, Arbeitsmarkt-Effekte schwächer.  
- **Nächster Schritt**: Mehr Modellflexibilität & fokussierte Feature-Selektion, um den durch Gewichtung angedeuteten Informationsgewinn besser auszuschöpfen.  

---


## **8. Naive Bayes Klassifikation**

Im nächsten Schritt wird ein Naive Bayes Modell verwendet, um die Richtung der Kursbewegung (steigend/fallend) zu klassifizieren.

Hier führen wir eine Klassifikation durch, ob der Kurs steigt oder fällt. Als Input werden alle verfügbaren Features genutzt.


In [ ]:
nb_model, X_tr, X_te, y_tr, y_te = naive_bayes(
    X, y_classification, train_ratio=0.7
)
evaluate_classification(nb_model, X_tr, y_tr, X_te, y_te)


### Interpretation  

##### 1 ) Performance-Übersicht  

| Datensatz | Accuracy | Precision&nbsp;0 / 1 | Recall&nbsp;(Sensitivität)&nbsp;0 / 1 | F1-Score&nbsp;0 / 1 | Klassen­verhältnis |
|-----------|---------:|----------------------|---------------------------------------|--------------------|--------------------|
| **Train** | **0.53** | 0.53 / 0.54 | 0.49 / 0.58 | 0.51 / 0.55 | 1341 : 1380 |
| **Test**  | **0.51** | 0.50 / 0.51 | 0.46 / 0.55 | 0.48 / 0.53 | 575 : 592 |

**Relevanz (Precision)**  
- Misst den Anteil der **korrekten positiven Vorhersagen** an allen „steigt“-Calls.  
- Wert ~0.51 auf dem Test-Set ⇒ Von 100 Long-Signalen sind knapp 51 tatsächlich steigende Perioden ⇒ jede zweite Position wäre unnötig.  

**Sensitivität (Recall)**  
- Misst den Anteil der **erkannten positiven Fälle** an allen tatsächlich steigenden Perioden.  
- Recall(1) = 0.55 ist höher als Recall(0) = 0.46 → Modell erkennt gut die Hälfte aller realen Anstiege, übersieht aber fast ebenso viele.  
- Geringer Recall(0) bedeutet, dass fast **54 %** der echten Rückgänge als „steigt“ fehlklassifiziert werden – kritisch, wenn Short-Side-Risiken abgesichert werden sollen.  

> **Gesamtbild:** Sowohl Relevanz als auch Sensitivität liegen knapp über dem Zufalls­niveau. Keines der beiden Qualitäts­kriterien ist in einem Trading-Kontext ausreichend, da Fehl­signale Transaktions­kosten und Drawdowns verursachen.

##### 2 ) Confusion-Matrix-Einordnung  

|                 | **Vorhergesagt 0** | **Vorhergesagt 1** |
|-----------------|-------------------:|-------------------:|
| **Tatsächlich 0** | 266 (Test) / 653 (Train) | 309 / 688 |
| **Tatsächlich 1** | 268 / 586 | 324 / 794 |

- **Fehler­symmetrie**: Falsch-Positiv- und Falsch-Negativ-Raten sind ähnlich hoch.  
- **Bias Richtung 1**: Das Modell bevorzugt Long-Signale ⇒ Recall(1) > Recall(0).  

##### 3 ) Warum performt Naive Bayes schwach?  

1. **Unabhängigkeits­annahme verletzt** – News-Features, Volumen & Returns sind korreliert.  
2. **Verteilungsform** – Gaussian-NB passt schlecht zu heavy-tailed Finanzdaten.  
3. **Kurzes Prognosefenster (30 min)** – Grundrauschen dominiert, Signal-to-Noise extrem klein.  
4. **Feature-Set** – identisch zur Regression; diskrete Schwellen oder Interaktionen fehlen.  

##### 4 ) Handlungsempfehlungen  

| Hebel | Idee |
|-------|------|
| **Baselines austauschen** | Regularisierte logistische Regression. |
| **Nicht-lineare Modelle** | Gradient Boosting, Random Forest, SVM, MLP. |
| **Feature Engineering** | Binning von Impact-Scores, Interaktion *Event-Typ × Impact*, Momentum-Indikatoren. |
| **Kostensensitive Lernziele** | Verlustmatrix an Long/Short-Kosten anpassen, Entscheidungs­schwelle optimieren. |
| **Kalibrierte Wahrscheinlichkeiten** | Platt/Isotonic Scaling, um Trades nur bei ≥ p* auszulösen. |

##### 5 ) TL;DR  

- **Accuracy & F1 ≈ 0.5** ⇒ Modell kaum besser als Münzwurf.  
- **Relevanz ~ Sensitivität** zeigen, dass jedes zweite Signal falsch ist und gut die Hälfte der wahren Bewegungen verpasst wird.  
- Kernprobleme: Naive-Bayes-Annahmen + geringes Richtungs­signal. Nicht-lineare Modelle und feature-reichere Repräsentationen sind nötig, um jenseits der 50-%-Schwelle zu kommen.

---


## **9. Lineare Diskriminanzanalyse (LDA)**

Hier wird eine LDA durchgeführt, um die Kursrichtung ebenfalls zu klassifizieren und die Ergebnisse mit dem Naive Bayes Modell zu vergleichen.

In [ ]:
lda_model, X_tr2, X_te2, y_tr2, y_te2 = lda(
    X, y_classification, train_ratio=0.7
)
evaluate_classification(lda_model, X_tr2, y_tr2, X_te2, y_te2)


### Interpretation  

##### 1 ) Performance‐Übersicht  

| Datensatz | Accuracy | Precision&nbsp;(Relevanz)&nbsp;0 / 1 | Recall&nbsp;(Sensitivität)&nbsp;0 / 1 | F1-Score&nbsp;0 / 1 | Klassen­verhältnis |
|-----------|---------:|--------------------------------------|---------------------------------------|--------------------|--------------------|
| **Train** | **0.55** | 0.55 / 0.55 | 0.46 / 0.62 | 0.50 / 0.58 | 1341 : 1380 |
| **Test**  | **0.51** | 0.50 / 0.52 | 0.44 / 0.58 | 0.47 / 0.55 | 575 : 592 |

###### Relevanz (Precision)  
- **Class 0**: 0.50 → Von 100 „Fall“-Signalen sind nur 50 korrekt.  
- **Class 1**: 0.52 → Etwas bessere Trefferquote für „Steigt“, aber jedes zweite Long-Signal bleibt Fehltrade.

###### Sensitivität (Recall)  
- **Class 1 (0.58)** > **Class 0 (0.44)**. LDA erkennt ~58 % aller realen Anstiege, lässt aber 42 % unentdeckt.  
- Lediglich 44 % der tatsächlichen Rückgänge werden korrekt erkannt → Short-Chancen gehen oft verloren.

> **Bottom line:** Trotz leichter Verbesserung auf dem Trainings-Set (Acc 55 %) liegt die **Test-Accuracy bei 51 %**, kaum höher als Zufall. Relevanz und Sensitivität verharren um die 0.5-Marke – für profitables Intraday-Trading unzureichend.

##### 2 ) Confusion‐Matrix (Test-Set)  

|                 | **Prädi­ziert 0** | **Prädi­ziert 1** |
|-----------------|------------------:|------------------:|
| **Tatsäch­lich 0** | **251** (True Neg) | **324** (False Pos) |
| **Tatsäch­lich 1** | **247** (False Neg) | **345** (True Pos) |

- **Bias zu Klasse 1**: Mehr False-Positives als False-Negatives → Modell tendiert zu Long-Calls.  
- **Fehler­quote**: 571 von 1167 Beobachtungen sind Fehlklassifikationen (≈ 49 %).

##### 3 ) Vergleich zu Naive Bayes  

| Modell | Train-Acc | Test-Acc | Recall(1) | Hauptunterschied |
|--------|----------:|---------:|-----------:|------------------|
| **Naive Bayes** | 0.53 | 0.51 | 0.55 | Unabhängigkeits­annahme, Gaussian-Dichte |
| **LDA** | **0.55** | 0.51 | **0.58** | Nutzt Kovarianzstruktur → etwas mehr „Steigt“ erkannt, aber Generalisierung nicht besser |

> **Interpretation:** LDA eliminiert die starke Unabhängigkeits­annahme von NB und passt Mittelwert-Vektoren & gemeinsame Kovarianz an. Das bringt **+2 pp** Accuracy im Train-Set, **aber keinen Test-Gewinn** → Hinweis auf leichte Überanpassung und insgesamt schwaches Signal im Datensatz.

##### 4 ) Warum bleibt die Leistung limitiert?  

1. **Fast komplett überlappende Klassen** – 30-Min-Returns nähern sich einer zufälligen Binär­verteilung.  
2. **Lineare Entscheidungsgrenze** – Falls die Trennung im Feature-Raum nicht linear ist, kann LDA sie nicht erfassen.  
3. **Homoskedastizitätsannahme** – LDA nimmt gleiche Kovarianz­matrix für beide Klassen an; bei hetero­skedastischen Finanzdaten unrealistisch.  
4. **Feature-Set** – Identisch zum Regressionsmodell; keine speziell auf Richtung optimierten Signale (Momentum-Vorzeichen, Schwellen­indikatoren …).

##### 5 ) Handlungsempfehlungen  

| Bereich | Maßnahme |
|---------|----------|
| **Modell** | Quadratische DA (QDA) zulassen, Gradient Boosting, XGBoost, Random Forest, SVM mit RBF-Kernel. |
| **Regularisierung** | Shrinkage-LDA oder Ridge-Logit testen, um Varianz zu reduzieren. |
| **Feature Engineering** | Diskretisiertes „Impact > x“-Flag, gleitende Vorzeichenwechsel, Interaktion *Event-Typ × Impact*. |
| **Threshold-Tuning** | Statt fester 0.5-Schwelle kosten­sensitives ROC-/PR-Optimieren auf Sharpe Ratio oder Hit-Rate. |
| **Ensemble** | Combine directional classifier with volatility filter (only trade when |predicted return| > k·σ). |

##### 6 ) TL;DR  

- **LDA** schlägt Naive Bayes im Train-Set, **nicht** im Test-Set (Acc bleibt 51 %).  
- Leichter **Long-Bias**, Relevanz & Sensitivität um 0.5 – praktisch Münzwurf.  
- Limitierende Faktoren: Lineare Trennbarkeit, homoskedastische Annahme, geringes Richtungs­signal.  
- **Nicht-lineare, regularisierte oder ensemble-basierte Ansätze** + gezieltes Feature-Engineering sind nötig, um die 50 %-Barriere zu überwinden.

---


## **10. Logistische Regression**

Hier wird eine LDA durchgeführt, um die Kursrichtung ebenfalls zu klassifizieren und die Ergebnisse mit dem Naive Bayes Modell zu vergleichen.

In [ ]:
logreg_model, X_tr3, X_te3, y_tr3, y_te3 = logistic_regression_classifier(
    X,                      
    y_classification,        
    train_ratio=0.7         
)

evaluate_classification(logreg_model, X_tr3, y_tr3, X_te3, y_te3)

### Logistische Regression – Interpretation  

##### 1 ) Performance-Übersicht  

| Datensatz | Accuracy | Precision (Relevanz) 0 / 1 | Recall (Sensitivität) 0 / 1 | F1-Score 0 / 1 | Klassen­verhältnis |
|-----------|---------:|---------------------------|-----------------------------|-----------------|--------------------|
| **Train** | **0.53** | 0.53 / 0.53 | 0.42 / 0.65 | 0.47 / 0.59 | 1341 : 1380 |
| **Test**  | **0.54** | 0.54 / 0.54 | 0.43 / 0.64 | 0.48 / 0.58 | 575 : 592 |

**Relevanz (Precision)**  
- *Class 0* = 0.54 → Nur 54 % der „Fall“-Signale korrekt.  
- *Class 1* = 0.54 → Gleich hohe Treffer­quote für „Steigt“.  

**Sensitivität (Recall)**  
- *Class 1* = 0.64 → Modell findet knapp zwei Drittel aller realen Anstiege.  
- *Class 0* = 0.43 → Erkennt weniger als die Hälfte der tatsächlichen Rückgänge.  

> **Gesamtbild:** Mit **54 % Accuracy** schlägt die logistische Regression Naive Bayes und LDA um ~3 Prozentpunkte, bleibt aber nur knapp über Zufall.


##### 2 ) Confusion-Matrix (Test)  

|                 | **Pred 0** | **Pred 1** |
|-----------------|-----------:|-----------:|
| **True 0**      | **249**    | **326** |
| **True 1**      | **216**    | **376** |

- **Long-Bias:** 326 False-Positives vs. 216 False-Negatives → Modell tendiert zu Long-Calls.  
- **Fehler­rate:** 542 von 1167 Beobachtungen sind Fehl­klassifikationen (≈ 46 %).  


##### 3 ) Vergleich zu bisherigen Modellen  

| Modell | Test-Accuracy | Recall(1) | Recall(0) | Haupteindruck |
|--------|--------------:|----------:|----------:|---------------|
| Naive Bayes | 0.51 | 0.55 | 0.46 | Fast Zufall, starke Annahme­verletzungen |
| LDA | 0.51 | 0.58 | 0.44 | Linear, homoskedastisch, kaum Generalisierung + |
| **Logit** | **0.54** | **0.64** | 0.43 | Beste Recall (1), aber niedrige Recall (0); leichter Fortschritt |

> **Interpretation:** Die regularisierte logistische Regression profitiert davon, Feature-Korrelationen zuzulassen und eine probabilistische Schwelle zu optimieren. Dennoch bleibt das Richtungs­signal schwach.


##### 4 ) Warum bremst die Performance?  

1. **Klassenüberlappung:** Steigen/Fallen in 30 min ist größtenteils zufällig – Signal-to-Noise gering.  
2. **Lineare Entscheidungs­grenze:** Logit erfasst keine nicht-linearen Interaktionen zwischen News-Impact und Markt­zustand.  
3. **Asymmetrische Kosten nicht berücksichtigt:** Falsche Long-Signale (FP) können teuer sein, werden hier gleich gewichtet.  
4. **Unverändertes Feature-Set:** Noch immer rein numerische Aggregationen; keine Schwellen- oder Richtungs­­indikatoren.  


##### 5 ) Handlungsempfehlungen  

| Bereich | Maßnahme |
|---------|----------|
| **Schwellen­optimierung** | ROC/PR-Kurven analysieren, Schwelle > 0.5 an Kosten/Nutzen anpassen. |
| **Nicht-lineare Modelle** | Gradient Boosting (XGBoost, LightGBM), Random Forest, SVM-RBF, MLP. |
| **Feature Engineering** | Diskrete Flags (Impact > x), Momentum-Vorzeichen, Interaktion *Event-Typ × Impact*, Intraday-Seasonality. |
| **Kostensensitives Lernen** | Custom Loss, Gewichtung FP vs. FN entsprechend Handels­strategie. |
| **Ensembles/Stacking** | Kombinieren von Logit-Wahrscheinlichkeiten mit Volatilitäts-Regressor → nur traden, wenn |pred_return| über Schwellwert liegt. |



##### 6 ) TL;DR  

- **Logistische Regression verbessert Accuracy auf 54 %** – bestes Ergebnis der drei Grundlagen­modelle, aber weiterhin nur knapp über Münzwurf.  
- **Recall (1) = 0.64** → erkennt die Mehrzahl der Anstiege, übersieht jedoch viele Rückgänge und produziert viele Long-Fehltrades.  
- **Linearität & Feat**

---
